# Лабораторные работы 1-3. Замеры времени вычисления

## ЛР 1 и 2

In [1]:
def integrate(f, a: float, b: float, *, n_iter: int = 1000):
    if a == b:
        return 0

    # вычисление по методу трапеций
    h = (b - a) / n_iter
    s = (f(a) + f(b)) / 2
    x = a + h
    
    for i in range(int(n_iter)-1):
        s += f(x + i*h)
    
    result = h * s
    return float(round(result, 8))

In [2]:
import math

%timeit -n100 -r10 integrate(math.atan, 0, math.pi / 2, n_iter=10**4)
%timeit -n100 -r10 integrate(math.atan, 0, math.pi / 2, n_iter=10**5)
%timeit -n100 -r10 integrate(math.atan, 0, math.pi / 2, n_iter=10**6)

1.47 ms ± 208 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
13.4 ms ± 121 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
136 ms ± 3.61 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [3]:
import concurrent.futures as ftres

def integrate_async(f, a: float, b: float, *, n_jobs: int = 2, n_iter: int = 1000):
    executor = ftres.ThreadPoolExecutor(max_workers=n_jobs) # потоки
    step = (b - a) / n_jobs # шаг вычисления интеграла, кол-во частей равно кол-ву потоков

    fs = [(a + i * step, a + (i + 1) * step) for i in range(n_jobs)] # интервалы для каждого потока
    spawn_lst = [executor.submit(integrate, f, *interval, n_iter=n_iter // n_jobs) for interval in fs] # запуск вычислений в потоках, кол-во итераций n_iter делится на кол-во потоков
    s = [r.result() for r in ftres.as_completed(spawn_lst)] # сохранение значений интерграла на интервалах
    
    return sum(s) # значение интеграла - сумма значений на каждом интервале, вычисленном в отдельном потоке

In [4]:
# n_jobs = 2
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**6)

2.43 ms ± 79.4 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
18.3 ms ± 550 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
241 ms ± 5.71 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [5]:
# n_jobs = 4
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**6)

3.05 ms ± 59.6 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
19.5 ms ± 410 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
262 ms ± 3.69 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [6]:
# n_jobs = 6
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**6)

3.13 ms ± 153 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
20.1 ms ± 487 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
257 ms ± 3.17 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


## ЛР 3. Cython

In [7]:
%load_ext Cython

In [8]:
%%cython

from cython.parallel import prange # параллельный range
from libc.math cimport atan # функция стандартной библиотеки C


ctypedef double (*func)(double x) nogil # объявление типа функции (аргумент f функции integrate)
# nogil - выключен GIL, т.к. из integrate - nogil-функции - нельзя вызвать функцию с включённым GIL


cdef double integrate(func f, float a, float b, int n_iter = 1000) nogil:
    # GIL выключен, вычисление производится параллельно
    # тип double - для повышения точности вычисления
    # f - Cython-функция, т.к. из Cython-функции нельзя вызвать функцию Python
    
    # значение интеграла на интервале, имеющем длину 0, равно 0
    if a == b:
        return 0

    # вычисление по методу трапеций
    cdef double s = (f(a) + f(b)) / 2
    cdef double h = (b - a) / n_iter
    cdef double x = a + h
    
    cdef int i
    for i in prange(n_iter - 1):
        s += f(x + i * h)
        
    cdef double result = h * s
    
    return result


cpdef integrate_function(a, b, n_iter):
    # функцию cpdef можно вызвать из функций Python и Cython
    # сделана для timeit
    # нельзя вынести в другую ячейку, т.к. функцию Cython можно вызвать только в той же ячейке
    return integrate(atan, a, b, n_iter)


In [9]:
%timeit -n100 -r10 integrate_function(0, math.pi / 2, n_iter=10**4)
%timeit -n100 -r10 integrate_function(0, math.pi / 2, n_iter=10**5)
%timeit -n100 -r10 integrate_function(0, math.pi / 2, n_iter=10**6)

236 µs ± 2.51 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
1.91 ms ± 34.5 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
18.8 ms ± 570 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


## ЛР 3. Joblib

In [10]:
# функция из ЛР 1
def integrate(f, a: float, b: float, *, n_iter: int = 1000):
    if a == b:
        return 0

    # вычисление по методу трапеций
    h = (b - a) / n_iter
    s = (f(a) + f(b)) / 2
    x = a + h
    
    for i in range(int(n_iter)-1):
        s += f(x + i*h)
    
    result = h * s
    return float(round(result, 8))

In [11]:
from joblib import Parallel, delayed

def integrate_async(f, a: float, b: float, *, n_jobs: int = 2, n_iter: int = 1000, backend='threading'):
    step = (b - a) / n_jobs # шаг вычисления интеграла, кол-во частей равно кол-ву потоков

    with Parallel(n_jobs=n_jobs, backend=backend) as p:
        # каждый поток вычисляет значение интеграла на части интервала
        # кол-во итераций n_iter делится на кол-во потоков
        fs = (delayed(integrate)(f, a + i * step, a + (i + 1) * step, n_iter=n_iter // n_jobs)
              for i in range(n_jobs))
        return sum(p(fs)) # значение интеграла - сумма значений на каждом интервале, вычисленном в отдельном потоке


Потоки (backend=threading)

In [12]:
# n_jobs = 2
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**6)

3.96 ms ± 224 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
21.8 ms ± 201 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
259 ms ± 3.53 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [13]:
# n_jobs = 4
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**6)

4.37 ms ± 501 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
23.1 ms ± 449 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
277 ms ± 4.74 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [14]:
# n_jobs = 6
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**4)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**5)
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**6)

4.68 ms ± 336 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
23.9 ms ± 370 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
287 ms ± 3.68 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


Процессы (backend=mutliprocessing)

In [15]:
# n_jobs = 2
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**4, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**5, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_iter=10**6, backend='multiprocessing')

41.5 ms ± 1.17 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
54.1 ms ± 2.24 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
143 ms ± 2.54 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [16]:
# n_jobs = 4
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**4, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**5, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=4, n_iter=10**6, backend='multiprocessing')

50.5 ms ± 2.03 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
60.7 ms ± 2.88 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
114 ms ± 1.77 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [17]:
# n_jobs = 6
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**4, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**5, backend='multiprocessing')
%timeit -n100 -r10 integrate_async(math.atan, 0, math.pi / 2, n_jobs=6, n_iter=10**6, backend='multiprocessing')

61.4 ms ± 1.77 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
69.1 ms ± 995 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
108 ms ± 1.79 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
